In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
import pandas as pd
import data_process as dp

dp.pdb_to_3Di_csv("../data", "./data.csv")
df = pd.read_csv("./data.csv")

In [3]:
train, val = dp.get_train_val_split(df["3Di"].tolist(), ratio=0.9)
train, val

([[2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   12,
   2,
   12,
   12,
   2,
   2,
   12,
   2,
   2,
   2,
   12,
   12,
   12,
   12,
   2,
   2,
   17,
   12,
   12,
   2,
   12,
   12,
   12,
   12,
   12,
   12,
   17,
   12,
   12,
   12,
   17,
   17,
   12,
   12,
   12,
   17,
   12,
   12,
   12,
   12,
   2,
   2,
   12,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   12,
   12,
   2,
   12,
   12,
   12,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   12,
   2,
   2,
   12,
   17,
   11,
   17,
   17,
   17,
   17,
   17,
   17,
   13,
   13,
   17,
   17,
   1,
   17,
   17,
   17,
   17,
   6,
   2,
   15,
   12,
   12,
   15,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   17,
   12,
   12,
   12,
   

In [28]:
batch_size = 4
block_size = 8
X, Y = dp.get_batch(train, batch_size=batch_size, block_size=block_size)
print(X.shape, Y.shape) # (B, T)
print("X", X,"\nY", Y)

torch.Size([4, 8]) torch.Size([4, 8])
X tensor([[ 2,  2,  2,  2,  2, 12,  2, 12],
        [17, 17,  2,  6,  2,  9, 12,  2],
        [ 2,  2,  2, 12, 12,  2, 12, 12],
        [12, 12, 12, 12, 12, 12, 12,  2]]) 
Y tensor([[ 2,  2,  2,  2, 12,  2, 12, 12],
        [17,  2,  6,  2,  9, 12,  2,  9],
        [ 2,  2, 12, 12,  2, 12, 12,  2],
        [12, 12, 12, 12, 12, 12,  2,  2]])


In [73]:
# revised and copied from https://github.com/karpathy/ng-video-lecture?tab=readme-ov-file
class SimpleLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # (C, C)
    
    def forward(self, idx, targets=None):
        """idx is the context and is shape (B, T)"""

        # here we select out embeddings for all the tokens up to T
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # each index is the target of each row in logits
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = SimpleLM(len(dp.TOKENS))
logits, loss = m(X, Y)
print(logits.shape)
print(loss)

gen = m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=128)[0].tolist()
print(dp.decode(gen))

torch.Size([32, 20])
tensor(2.9805, grad_fn=<NllLossBackward0>)
AQHWRAQRYWATPNTMNYQKGRLMQERYGDGPYKNNKPEIQKDGHPWHSELQKAASCGHHCWQFFDLGSKLCSMWCGHSMSYQQLCNFRLCDWMWQPLYVTHNYQLQCTNWHWWIWMNSCAMSCDSKVM
